# $\color{blue}{\text{MADE}}$ $\bullet$ $\color{red}{\text{Advanced ML}}$ $\bullet$ HW №2
### Преподаватель: [*Сергей Николенко*](https://logic.pdmi.ras.ru/~sergey/index.html)
### Выполнил: $\text{Саит Шарипов, группа MADE-DS-22}$
### 🦉 EM-алгоритм и рейтинг-система спортивного ЧГК
* * *

# 📝 [Описание задания](https://docs.google.com/document/d/1MapaSpMax0cEnnY2-QJ2XMgzVHpj_UrucJK-1r5FgA8/edit):

Второе домашнее задание — самое большое в курсе, в нём придётся и концептуально подумать о происходящем, и технические трудности тоже порешать. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются.

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить **вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК)**.

**Background:** в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно $36$ или $45$, иногда $60$, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
* построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
* при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
* а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.

Я сделал за вас только первый шаг: выкачал через API [сайта рейтинга ЧГК](https://rating.chgk.info/) все нужные данные, чтобы сайт не прилёг под вашими многочисленными скрейперами. :) Полученные данные лежат в формате `pickle` вот здесь: 👉🏻 [ссылка на данные](https://www.dropbox.com/s/s4qj0fpsn378m2i/chgk.zip?dl=0).

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле `mask` в `results.pkl`). Для унификации предлагаю:
    * взять в тренировочный набор турниры с `dateStart` из $2019$ года; 
    * в тестовый — турниры с `dateStart` из $2020$ года.
2. Постройте `baseline`-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
    * повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
    * в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
    * для `baseline`-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.
3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
    * предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
    * в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров$^1$.
4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
    * предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
    * разработайте EM-схему для обучения этой модели, реализуйте её в коде;
    * обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.
5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в [базе вопросов ЧГК](https://db.chgk.info/) (это чисто техническое дело, тут никакого ML нету).
6. 🎁 *Бонус*: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. Скорее всего, вы увидите, что топ занят игроками, которые сыграли совсем мало вопросов, около $100$ или даже меньше; если вы поищете их в официальном рейтинге ЧГК, вы увидите, что это какие-то непонятные ноунеймы$^2$. В baseline-модели, скорее всего, такой эффект будет гораздо слабее.

    Это естественное свойство модели: за счёт EM-схемы влияние $1-2$ удачно сыгранных турниров будет только усиливаться, потому что неудачных турниров, чтобы его компенсировать, у этих игроков нет. Более того, это не мешает метрикам качества, потому что если эти игроки сыграли всего $1-2$ турнира в $2019$-м, скорее всего они ничего или очень мало сыграли и в $2020$, и их рейтинги никак не влияют на качество тестовых предсказаний. Но для реального рейтинга такое свойство, конечно, было бы крайне нежелательным. Давайте попробуем его исправить:
    * сначала жёстко: выберите разумную отсечку по числу вопросов, учитывая, что в одном турнире их обычно $30-50$;
    * можно ли просто выбросить игроков, которые мало играли, и переобучить модель? почему? предложите, как нужно изменить модель, чтобы не учитывать слишком мало сыгравших, и переобучите модель;
    * но всё-таки это не слишком хорошее решение: если выбрать маленькую отсечку, будут ноунеймы в топе, а если большую, то получится, что у нового игрока слишком долго не будет рейтинга; скорее всего, никакой “золотой середины” тут не получится;
    * предложите более концептуальное решение для топа игроков в рейтинг-листе; если получится, реализуйте его на практике (за это уж точно будут серьёзные бонусные баллы).
7. 🎁 *Бонус*: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем этого добиться:
    * если хватит вычислительных ресурсов, сначала сделайте `baseline` совсем без таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не только на турнирах $2019$ года; улучшилось ли качество предсказаний на $2020$?
    * одну схему со временем мы уже использовали: брали для обучения только последний год турниров; примерно так делают, например, в теннисной чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки (например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
    * предложите варианты базовой модели или алгоритма её обучения, которые могли бы реализовать изменения рейтинга со временем; если получится, реализуйте их на практике, проверьте, улучшатся ли предсказания на $2020$.
---
$^1$Для самопроверки: у меня средняя корреляция Спирмена на тестовом множестве $2020$ года во всех моделях, включая baselines, получалась порядка $0.7-0.8$, а корреляция Кендалла — порядка $0.5-0.6$. Если у вас корреляции вышли за $0.9$ или, наоборот, упали ниже $0.3$, скорее всего где-то баг.

$^2$Для самопроверки: а вот те игроки, кто сыграл от тысячи вопросов и больше и при этом всё равно попал в топ-$100$ весов модели, должны быть настоящими топовыми игроками из ведущих команд официального рейтинга. Если это не так, опять же, скорее всего где-то баг.

---






# ✅ Решение

## 0️⃣ Импорт библиотек

In [1]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

import numpy as np
import pandas as pd

import re
import pickle
import requests

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

from scipy import stats
from scipy.special import logit, expit

## 1️⃣ Чтение и анализ данных
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле `mask` в `results.pkl`). Для унификации предлагаю:
* взять в тренировочный набор турниры с `dateStart` из $2019$ года; 
* в тестовый — турниры с `dateStart` из $2020$ года.

<font color='Navy'><i> 
✏️ Читаем данные.
</i></font>

In [2]:
with open('chgk_data/results.pkl', 'rb') as fin:
    results = pickle.load(fin)
    
with open('chgk_data/tournaments.pkl', 'rb') as fin:
    tournaments = pickle.load(fin)
    
with open('chgk_data/players.pkl', 'rb') as fin:
    players = pickle.load(fin)

<font color='Navy'><i> 
✏️ Обработаем `tournaments`.
</i></font>

In [3]:
tournaments_df = pd.DataFrame(tournaments.values())
tournaments_df = tournaments_df.drop(['season', 'orgcommittee', 'synchData'], axis=1) # удалим лишние поля
tournaments_df.rename(columns={'name': 'tour_name'}, inplace=True)

# Преобразуем тип турнира в читаемый вид.
tour_type = tournaments_df['type'].apply(pd.Series)
tour_type = tour_type.drop(['id'], axis=1)
tournaments_df['type'] = tour_type['name']

# Дату в читаемый вид.
tournaments_df['dateStart'] = pd.to_datetime(tournaments_df['dateStart'], utc=True).dt.date
tournaments_df['dateEnd'] = pd.to_datetime(tournaments_df['dateEnd'], utc=True).dt.date
tournaments_df['year'] = pd.to_datetime(tournaments_df['dateStart']).dt.year

# Оставим только турниры из 2019 и 2020 года.
tournaments_df = tournaments_df[(tournaments_df['year'] == 2019) | (tournaments_df['year'] == 2020)]

tournaments_df.set_index('id')
tournaments_df

,id,tour_name,dateStart,dateEnd,type,questionQty,year
3788,4628,Семь сорок,2020-12-30,2020-12-30,Синхрон,"{'1': 12, '2': 12, '3': 12}",2020
3921,4772,Синхрон северных стран. Зимний выпуск,2019-01-05,2019-01-09,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
4100,4957,Синхрон Биркиркары,2020-02-20,2020-02-27,Синхрон,"{'1': 13, '2': 13, '3': 13}",2020
4115,4973,Балтийский Берег. 3 игра,2019-01-25,2019-01-29,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
4116,4974,Балтийский Берег. 4 игра,2019-03-01,2019-03-05,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
...,...,...,...,...,...,...,...
5523,6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05,2020-05-05,Обычный,"{'1': 12, '2': 12, '3': 12}",2020
5524,6482,Онлайн: 19:00 Зелёный шум,2020-05-07,2020-05-07,Обычный,"{'1': 13, '2': 13, '3': 13}",2020
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08,2020-05-08,Обычный,"{'1': 12, '2': 12, '3': 12}",2020
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04,2020-05-04,Обычный,"{'1': 12, '2': 12}",2020


In [4]:
tournaments_df['type'].unique()

array(['Синхрон', 'Асинхрон', 'Обычный', 'Строго синхронный',
       'Общий зачёт'], dtype=object)

In [5]:
tournaments_df[tournaments_df['type'] != 'Общий зачёт']

,id,tour_name,dateStart,dateEnd,type,questionQty,year
3788,4628,Семь сорок,2020-12-30,2020-12-30,Синхрон,"{'1': 12, '2': 12, '3': 12}",2020
3921,4772,Синхрон северных стран. Зимний выпуск,2019-01-05,2019-01-09,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
4100,4957,Синхрон Биркиркары,2020-02-20,2020-02-27,Синхрон,"{'1': 13, '2': 13, '3': 13}",2020
4115,4973,Балтийский Берег. 3 игра,2019-01-25,2019-01-29,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
4116,4974,Балтийский Берег. 4 игра,2019-03-01,2019-03-05,Синхрон,"{'1': 12, '2': 12, '3': 12}",2019
...,...,...,...,...,...,...,...
5523,6481,Онлайн: 15:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-05,2020-05-05,Обычный,"{'1': 12, '2': 12, '3': 12}",2020
5524,6482,Онлайн: 19:00 Зелёный шум,2020-05-07,2020-05-07,Обычный,"{'1': 13, '2': 13, '3': 13}",2020
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08,2020-05-08,Обычный,"{'1': 12, '2': 12, '3': 12}",2020
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04,2020-05-04,Обычный,"{'1': 12, '2': 12}",2020


<font color='Navy'><i> 
👆🏻 Видим, что таких турниров мало, но они длятся очень долго, судя по всему от них стоит избавится, чтобы не произошо дублирования с турнирами, которые участвуют в этом 'Общем зачёте'.
</i></font>

In [6]:
tournaments_df = tournaments_df[tournaments_df['type'] != 'Общий зачёт']

In [7]:
print(len(tournaments_df[tournaments_df['year'] == 2019]))
print(len(tournaments_df[tournaments_df['year'] == 2020]))

680
413


<font color='Navy'><i> 
✏️ Обработаем `players`. Соединим вместе имя, отчество и фамилию игроков.
</i></font>

In [8]:
players_df = pd.DataFrame(players.values())
players_df['patronymic'].fillna(value='_', inplace=True)
players_df['Name'] = (players_df['name'].str.cat(players_df['patronymic'], sep =' ')).str.cat(players_df['surname'], sep =' ')
players_df.pop('name')
players_df.pop('patronymic')
players_df.pop('surname')
players_df.rename(columns={'Name': 'name'}, inplace=True)

players_df

,id,name
0,1,Алексей _ Абабилов
1,10,Игорь Абалов
2,11,Наталья Юрьевна Абалымова
3,12,Артур Евгеньевич Абальян
4,13,Эрик Евгеньевич Абальян
...,...,...
204058,224700,Артём Евгеньевич Садов
204059,224701,Даниил Олегович Трефилов
204060,224702,Владимир Араратович Басенцян
204061,224703,Руслан Ринатович Дауранов


In [9]:
# Создадим словарь p_id -> p_name.
p_id_name_dict = {}
for p in players_df.values:
    p_id_name_dict[p[0]] = p[-1]

<font color='Navy'><i> 
✏️ Строчка ниже свидетельствует о том, что матчить `results` и `tournaments` надо по ключам `results` и полю `tournaments_df['id']` соответсвенно.
</i></font>

In [10]:
set(results.keys()) == set(list(tournaments_df['id']))

False

<font color='Navy'><i> 
✏️ Посмотрим, что бывает в `mask` кроме $1$ и $0$.
</i></font>

In [11]:
no_zero_one = set()
for tour_id, teams in tqdm(results.items()):
    for i, team in enumerate(teams):
        if 'mask' not in team or team['mask'] is None:
            continue
        for j in team['mask']:
            if j != '0' and j != '1':
                no_zero_one.add(j)
                
print(no_zero_one)

100%|█████████████████████████████████████| 5528/5528 [00:02<00:00, 2361.92it/s]

{'?', 'X'}


In [12]:
have_X = 0
have_question = 0
have_X_question = 0
no_mask = 0
have_mask = 0

for tour_id, teams in tqdm(results.items()):
    if tour_id in tournaments_df['id']:
        flag_have_max = True
        flag_no_question = True
        flag_no_X = True
        for i, team in enumerate(teams):
            if 'mask' not in team or team['mask'] is None:
                flag_have_max = False
            else:
                if '?' in team['mask']:
                    flag_no_question = False
                if 'X' in team['mask']:
                    flag_no_X = False
        no_mask += not(flag_have_max)
        have_mask += flag_have_max
        have_X_question += not(flag_no_question) and not(flag_no_X)
        have_question += not(flag_no_question)
        have_X += not(flag_no_X)

print("mask присутствует в {} турнирах".format(have_mask))
print("mask не присутствует в {} турнирах".format(no_mask))
print("В mask есть только ? в {} турнирах".format(have_question - have_X_question))
print("В mask есть только X в {} турнирах".format(have_X - have_X_question))
print("В mask есть ? и X в {} турнирах".format(have_X_question))

100%|████████████████████████████████████| 5528/5528 [00:00<00:00, 73378.19it/s]

mask присутствует в 1026 турнирах
mask не присутствует в 26 турнирах
В mask есть только ? в 1 турнирах
В mask есть только X в 108 турнирах
В mask есть ? и X в 2 турнирах


<font color='Navy'><i> 
👆🏻 Видим, что можем отбросить $3$ турнира, но зато не будем гадать, чем заполнить ? в `mask`. А что же делать с X?...
</i></font>

<font color='Navy'><i> 
✏️ Есть гипотеза, что X - это снятый вопрос. То есть судьи решили, что вопрос некорректный, и сняли для всех команд турнира. Проверим:
</i></font>

In [13]:
for tour_id, teams in tqdm(results.items()):
    i = 0
    flag_print = False
    if tour_id in tournaments_df['id']:
        for i, team in enumerate(teams):
            if 'mask' not in team or team['mask'] is None:
                break
            if 'X' in team['mask']:
                i += 1
                poss = [m.start() for m in re.finditer('X', team['mask'])]
                flag_print = True
                print("В  турнире c id={} в команде с id={} X в позициях {}".format(tour_id, team['team']['id'], poss))
                if i == 10:
                    break
    if flag_print:
        print()
        break

 60%|█████████████████████              | 3330/5528 [00:00<00:00, 208631.32it/s]

В  турнире c id=4150 в команде с id=6874 X в позициях [32]
В  турнире c id=4150 в команде с id=316 X в позициях [32]
В  турнире c id=4150 в команде с id=57024 X в позициях [32]
В  турнире c id=4150 в команде с id=641 X в позициях [32]
В  турнире c id=4150 в команде с id=4109 X в позициях [32]
В  турнире c id=4150 в команде с id=26956 X в позициях [32]
В  турнире c id=4150 в команде с id=41578 X в позициях [32]
В  турнире c id=4150 в команде с id=49804 X в позициях [32]
В  турнире c id=4150 в команде с id=414 X в позициях [32]
В  турнире c id=4150 в команде с id=6198 X в позициях [32]



<font color='Navy'><i> 
✏️ Гипотеза подтвердилась, поэтому `mask` с X можно не отбрасывать, просто не булем учитывать X.
</i></font>

In [14]:
mask_eq = 0
mask_diff = 0


for tour_id, teams in tqdm(results.items()):
    if tour_id in tournaments_df['id']:
        mask_lens = set()
        flag_have_mask = True
        for i, team in enumerate(teams):
            if 'mask' not in team or team['mask'] is None:
                flag_have_mask = False
                break
            mask_lens.add(len(team['mask']))
        if flag_have_mask:
            if len(mask_lens) > 1:
                mask_diff += 1
            else:
                mask_eq += 1
        
print("mask полная в {} турнирах.".format(mask_eq))
print("mask неполная в {} турнирах.".format(mask_diff))

100%|████████████████████████████████████| 5528/5528 [00:00<00:00, 75476.86it/s]

mask полная в 999 турнирах.
mask неполная в 27 турнирах.


<font color='Navy'><i> 
👆🏻 Видим, что иногда некоторые команды в турнире играли не столько же, сколько их соперники. Благо, что таких турниров не много и мы их можем отбросить.
</i></font>

In [15]:
good_tour_id = set(results.keys())
print("Изначально турниров: {}".format(len(good_tour_id)))


bad_tour_id = set()
for tour_id in good_tour_id:
    # Избавляемся от турниров не из 2019-2020 годов.
    if not tournaments_df['id'].isin([tour_id]).any():
        bad_tour_id.add(tour_id)
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры не из 2019-2020 года. Осталось турниров: {}".format(len(good_tour_id)))

bad_tour_id = set()
for tour_id in good_tour_id:
    # Избавляемся от турниров, в которых не указаны результаты.
    if len(results[tour_id]) == 0:
        bad_tour_id.add(tour_id)
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры, в которых не указаны результаты. Осталось турниров: {}".format(len(good_tour_id)))
        
    
bad_tour_id = set()
for tour_id in good_tour_id:
    teams = results[tour_id]
    for i, team in enumerate(teams):
        # Убираем турниры, в которых нет данных о составах команд.
        if len(team['teamMembers']) == 0:
            bad_tour_id.add(tour_id)
            break
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры в которых нет данных о составах команд. Осталось турниров: {}".format(len(good_tour_id)))


bad_tour_id = set()
for tour_id in good_tour_id:
    teams = results[tour_id]
    for i, team in enumerate(teams):
        # Убираем турниры, в которых нет данных о повопросных результатах
        if 'mask' not in team or team['mask'] is None:
            bad_tour_id.add(tour_id)
            break
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры в которых нет данных о повопросных результатах. Осталось турниров: {}".format(len(good_tour_id)))


bad_tour_id = set()
for tour_id in good_tour_id:
    teams = results[tour_id]
    for i, team in enumerate(teams):
        # Убираем турниры, в которых есть команда, у которой в маске есть ?
        if '?' in team['mask']:
            bad_tour_id.add(tour_id)
            break
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры в которых есть команда, у которой в маске есть ?. Осталось турниров: {}".format(len(good_tour_id)))


# Убираем турниры, в которых у команд разное количество ответов
bad_tour_id = set()
for tour_id in good_tour_id:
    teams = results[tour_id]
    mask_lens = set()
    for i, team in enumerate(teams):
        mask_lens.add(len(team['mask']))
    if len(mask_lens) > 1:
        bad_tour_id.add(tour_id)
good_tour_id = good_tour_id - bad_tour_id
print("Убрали турниры, в которых у команд разное количество ответов. Осталось турниров: {}".format(len(good_tour_id)))

Изначально турниров: 5528
Убрали турниры не из 2019-2020 года. Осталось турниров: 1093
Убрали турниры, в которых не указаны результаты. Осталось турниров: 864
Убрали турниры в которых нет данных о составах команд. Осталось турниров: 845
Убрали турниры в которых нет данных о повопросных результатах. Осталось турниров: 811
Убрали турниры в которых есть команда, у которой в маске есть ?. Осталось турниров: 796
Убрали турниры, в которых у команд разное количество ответов. Осталось турниров: 794


In [85]:
tours_id = tournaments_df['id'].values
tours_name = tournaments_df['tour_name'].values

tour_id_name_dict = {}

for t, n in zip(tours_id, tours_name):
    tour_id_name_dict[t] = n

In [16]:
# Создаем словарь, в котором для каждого турнира будем хранить год, чтобы быстро обращаться
dict_tour_id_year = {}
for t in tournaments_df.values:
    dict_tour_id_year[t[0]] = t[-1]

In [69]:
tour_team_question_df = {
    "year": [],
    "tour_id": [],
    "teams_id": [],
    "qustions_id": [],
}

team_members_df = {
    "year": [],
    "team_id": [],
    "team_name": [],
    "players_id": [],
}

df = {
    'year': [], # year
    'tour_id': [], # tour
    't_id': [], # team
    'p_id': [], # player
    'q_id': [], # question
    'y': [],    # Did team answer?
}

tour_id_to_teams_id_dict20 = {}
team_id_to_players_id_dict20 = {}
tour_id_to_teams_results_dict20 = {}
tour_id_to_questions_dict19 = {}


q_id_cnt = 0

for tour_id in tqdm(good_tour_id):
    teams_id = []
    year_tour = dict_tour_id_year[tour_id]
    if year_tour == 2020:
        tour_id_to_teams_results_dict20[tour_id] = {}
    list_of_teams = results[tour_id]
    flag_first_team = True
    qustions_id = []
    for team in list_of_teams:
        players_id = []
        team_id = team['team']['id']
        teams_id.append(team_id)
        team_name = team['team']['name']
        team_mask = team['mask']
        team_mask = team_mask.replace('X', '') # убрали все X из маски
        team_members = team['teamMembers']
        team_res = 0
        
        # Cоставление id вопросов турнира:
        if flag_first_team:
            for m in team_mask:
                q_id_cnt += 1
                qustions_id.append(q_id_cnt)
                
            flag_first_team = False

        for m in team_mask:
            team_res += (1 - int(m == '0'))
        
        # Для каждого члена команды    
        for member in team_members:
            player_id = member['player']['id']
            players_id.append(player_id)
            
            # Для каждого вопроса в маске
            for i, q_id in enumerate(qustions_id): 
                df['tour_id'].append(tour_id)
                df['year'].append(year_tour)
                df['t_id'].append(team_id)
                df['p_id'].append(player_id)
                df['q_id'].append(q_id)
                df['y'].append(1 - int(team_mask[i] == '0'))
        
        team_members_df["year"].append(year_tour)
        team_members_df["team_id"].append(team_id)
        team_members_df["team_name"].append(team_name)
        team_members_df["players_id"].append(players_id)
        if year_tour == 2020:
            team_id_to_players_id_dict20[team_id] = players_id
            tour_id_to_teams_results_dict20[tour_id][team_id] = team_res
        
        
        
    tour_team_question_df["year"].append(year_tour)
    tour_team_question_df["tour_id"].append(tour_id)
    tour_team_question_df["teams_id"].append(teams_id)
    if year_tour == 2020:
        tour_id_to_teams_id_dict20[tour_id] = teams_id
    tour_team_question_df["qustions_id"].append(qustions_id)
    if year_tour == 2019:
        tour_id_to_questions_dict19[tour_id] = qustions_id
tour_team_question_df = pd.DataFrame(tour_team_question_df)
team_members_df = pd.DataFrame(team_members_df)
df = pd.DataFrame(df)

100%|█████████████████████████████████████████| 794/794 [00:15<00:00, 51.74it/s]


In [18]:
df.head()

,year,tour_id,t_id,p_id,q_id,y
0,2019,6144,27254,27469,1,1
1,2019,6144,27254,27469,2,0
2,2019,6144,27254,27469,3,1
3,2019,6144,27254,27469,4,0
4,2019,6144,27254,27469,5,0


In [19]:
team_members_df.head()

,year,team_id,team_name,players_id
0,2019,27254,Altavista,"[27469, 57286, 155103, 41104, 57288, 155102]"
1,2019,55270,МГТУ - Легионеры,"[147450, 139991, 132228, 100148, 176349, 83428]"
2,2019,55015,Lorem Ipsum,"[17686, 26243, 73105, 132648, 116892, 133839]"
3,2019,51228,Одиночество в SETI,"[12511, 95403, 22635, 41845, 69244, 34796]"
4,2019,46099,Команда Дмитрия Слоуща,"[29521, 631, 159638, 92631, 33935, 25113]"


In [20]:
tour_team_question_df.head()

,year,tour_id,teams_id,qustions_id
0,2019,6144,"[27254, 55270, 55015, 51228, 46099, 76244, 267...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,2019,6145,"[57542, 76272, 76288, 65319, 28163, 76282, 613...","[37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 4..."
2,2019,6146,"[46916, 48762, 65697, 55270, 76470, 70840, 758...","[82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 9..."
3,2019,6147,"[72610, 66481, 72773, 66687, 73568, 70296, 735...","[118, 119, 120, 121, 122, 123, 124, 125, 126, ..."
4,2019,6149,"[60432, 27269, 53714, 67543, 175, 167, 27610, ...","[166, 167, 168, 169, 170, 171, 172, 173, 174, ..."


<font color='Navy'><i> 
✏️ Делим данные на train и test:
</i></font>

In [21]:
train_df = df[df['year'] == 2019]
test_df = df[df['year'] == 2020]

In [22]:
train_df.drop(columns=['year'], inplace=True)
test_df.drop(columns=['year'], inplace=True)

In [23]:
train_df

,tour_id,t_id,p_id,q_id,y
0,6144,27254,27469,1,1
1,6144,27254,27469,2,0
2,6144,27254,27469,3,1
3,6144,27254,27469,4,0
4,6144,27254,27469,5,0
...,...,...,...,...,...
18165400,6141,27115,40360,36300,0
18165401,6141,27115,40360,36301,0
18165402,6141,27115,40360,36302,0
18165403,6141,27115,40360,36303,0


## 2️⃣ Baseline на основе логистической регрессии
Постройте `baseline`-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для `baseline`-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

<font color='Navy'><i> 
▪️ С помощью логистической регрессии будем предсказывать вероятность $\mathbb{P}(p_i \text{ ответил на }q_j)$.  
▪️ Будем считать, что если на вопрос ответила команда, то ответили все её игроки.  
▪️ В качестве признаком будем использовать ohe-векторы игрока и вопроса.  
▪️ После обучения модели веса перед игроками будут символизировать их силу f (force), а веса перед вопросами будут символизировать их сложность d (difficulty).  
▪️ Учет того, что в разных турнирах вопросы совсем разного уровня сложности, будет достигаться ровно за счёт того, что некоторые игроки ответили на данный вопрос, а некоторые нет и таким образом будет выставлена оптимальная сложность вопроса.  
▪️ При этом нельзя считать, что сложность вопроса есть доля команд, которые на него не ответили, поскольку как сказано выше турниры имеют разную сложность. 
</i></font>

### 2.1. One hot encoding для id игроков и вопросов

In [24]:
# указываем handle_unknown='ignore', так как в общем случае в тесте могут быть категории, которые мы не знаем
ohe = OneHotEncoder(handle_unknown='ignore')

X_train = ohe.fit_transform(train_df[['p_id', 'q_id']])
X_test = ohe.transform(test_df[['p_id', 'q_id']])

y_train = train_df['y']
y_test = test_df['y']

### 2.2. Обучение логистической регрессии

In [25]:
logreg_model = LogisticRegression(solver="saga")
logreg_model.fit(X_train, y_train)

LogisticRegression(solver='saga')

<font color='Navy'><i> 
✏️ Достаем силу игроков и сложность вопросов из весов логистической регрессии
</i></font>


In [26]:
len(train_df.p_id.unique()) + len(train_df.q_id.unique()) == len(logreg_model.coef_[0])

True

In [27]:
players_force = logreg_model.coef_[0][:len(train_df.p_id.unique())]
questions_difficulty = logreg_model.coef_[0][len(train_df.p_id.unique()):]

### 2.3. top-$100$ игроков из весов baseline-модели 

In [28]:
top_k = 100

players_rating_baseline = []

player_id_to_player_force_dict = {}

for i, p_id in tqdm(enumerate(ohe.categories_[0])):
    p_name = p_id_name_dict[p_id]
    p_force = players_force[i]
    player_id_to_player_force_dict[p_id] = p_force
    players_rating_baseline.append({'p_id': p_id, 'p_name': p_name, 'p_force': p_force})
    
players_rating_baseline = sorted(players_rating_baseline, key=lambda d: d['p_force'], reverse=True)
players_rating_baseline = pd.DataFrame(players_rating_baseline)
players_rating_baseline['pred_rating_pos'] = players_rating_baseline.index + 1

short_players_rating_baseline = players_rating_baseline[:top_k]

53398it [00:00, 252595.91it/s]


In [29]:
def get_real_rating_position(p_id):
    """По id ирока получаем его текущуюю позицию в рейтинге ЧГК на 2020 год."""
    url = "https://rating.chgk.info/api/players/{}/rating".format(p_id)
    try:
        js = requests.get(url).json()
        for x in js:
            if x['date'][:4] == '2020':
                true_rating_pos = int(x['rating_position'])
                break
    except:
        true_rating_pos = -1
    
    return true_rating_pos

short_players_rating_baseline['true_rating_pos'] = short_players_rating_baseline['p_id'].apply(get_real_rating_position)
short_players_rating_baseline[:20]

,p_id,p_name,p_force,pred_rating_pos,true_rating_pos
0,27403,Максим Михайлович Руссо,4.067195,1,5
1,4270,Александра Владимировна Брутер,3.936101,2,6
2,28751,Иван Николаевич Семушин,3.880612,3,2
3,27822,Михаил Владимирович Савченков,3.790631,4,3
4,30152,Артём Сергеевич Сорожкин,3.723719,5,1
5,30270,Сергей Леонидович Спешков,3.711229,6,4
6,20691,Станислав Григорьевич Мереминский,3.593963,7,96
7,18036,Михаил Ильич Левандовский,3.541633,8,28
8,22799,Сергей Игоревич Николенко,3.466650,9,16
9,26089,Ирина Сергеевна Прокофьева,3.464489,10,34


In [30]:
tmp = len(short_players_rating_baseline[short_players_rating_baseline['true_rating_pos'] <= 100])
print("{} человек из реального top-100 попали в baseline top-100".format(tmp))

46 человек из реального top-100 попали в baseline top-100


## 3️⃣ Качество baseline рейтинг-системы 
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров$^1$.
---
$^1$Для самопроверки: у меня средняя корреляция Спирмена на тестовом множестве $2020$ года во всех моделях, включая baselines, получалась порядка $0.7-0.8$, а корреляция Кендалла — порядка $0.5-0.6$. Если у вас корреляции вышли за $0.9$ или, наоборот, упали ниже $0.3$, скорее всего где-то баг.

<font color='Navy'><i> 
▪️ Будем считать, что рейтинг команды = среднему рейтингу игроков в ней.  
▪️ Ранжирование команд осуществим по рейтингу команд, вычисленному таким образом.  
▪️ Реальное ранжирование команд осуществим по числу отвеченных вопросов.  
▪️ Для каждого турнира вычислим ранговые корреляции Спирмена и Кендалла между реальным ранжированием команд в турнире и предсказанным моделью.  
▪️ Усредним вычсиленные корреляции по турнирам.
</i></font>

In [31]:
spearman_corrs = []
kendall_corrs = []

for tour_id, teams in tqdm(tour_id_to_teams_id_dict20.items()):
    pred_ratings = []
    true_ratings = []
    for team_id in teams:
        true_res = tour_id_to_teams_results_dict20[tour_id][team_id]
        pred_res = 0
        players_cnt = 0
        for p_id in team_id_to_players_id_dict20[team_id]:
            if p_id in player_id_to_player_force_dict:
                pred_res += player_id_to_player_force_dict[p_id]
                players_cnt += 1
        if players_cnt == 0:
            continue
        pred_res /= players_cnt
        pred_ratings.append(pred_res)
        true_ratings.append(true_res)
        
    spearman_corrs.append(stats.spearmanr(true_ratings, pred_ratings).correlation)
    kendall_corrs.append(stats.kendalltau(true_ratings, pred_ratings).correlation)
        
print("Корреляция Спирмена для baseline модели: {}".format(np.mean(np.array(spearman_corrs))))
print("Корреляция Кендалла для baseline модели: {}".format(np.mean(np.array(kendall_corrs))))

100%|████████████████████████████████████████| 155/155 [00:00<00:00, 325.14it/s]

Корреляция Спирмена для baseline модели: 0.7589100764463393
Корреляция Кендалла для baseline модели: 0.5995266595237723


## 4️⃣ EM-схема
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
   * предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
   * разработайте EM-схему для обучения этой модели, реализуйте её в коде;
   * обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

<font color='Navy'><i> 
▪️ Пусть $res_{tq}$ - это ответ команды t на вопрос q (для train мы это знаем).  
▪️ Пусть скрытая переменная $z_{pq}$ - это $\mathbb{P}(\text{игрок }p \text{ ответил на вопрос }q)$.   
▪️ **E шаг:** Вычисляем $$ \mathbb{E}[z_{pq}] =
  \begin{cases}
    \mathbb{P}(z_{pq} = 1 \mid res_{tq} = 1)       & \quad \text{если } res_{tq} = 1\\
    0  & \quad \text{если } res_{tq} = 0
  \end{cases}
$$  
где $$\mathbb{P}(z_{pq} = 1 \mid res_{tq} = 1) = \frac{\mathbb{P}(\text{игрок }p \text{ ответил на вопрос }q)}{1 - \prod_{\text{сокомандники } p}\big(1 - \mathbb{P}(\text{сокомандник} \text{ ответил на вопрос }q)\big)}$$ 
А эти вероятности мы как раз предсказали с помощью логистической регрессии в M шаге.  
▪️ **M шаг:** обучаем логистическую регрессию (как в baseline) предсказывать мягкую метку $\mathbb{E}[z_{pq}]$.  
В качестве начального приближения параметров можем взять baseline. 
</i></font>

In [90]:
# Начальная инициализация.
step_k = 5
eps = 1e-8
em_model = LinearRegression()
em_train_df = train_df.copy(deep=True)
em_train_df.drop(columns=['y'], inplace=True)
em_train_df['pred_for_player'] = logreg_model.predict_proba(X_train)[:, 1]
best_metric = 0
best_model = None 

In [91]:
for i in range(1, step_k + 1):

    # E шаг.
    em_train_df['invers_pred_for_player'] = 1 - em_train_df['pred_for_player']
    tmp = 1 - em_train_df.groupby(['tour_id', 't_id', 'q_id'])['invers_pred_for_player'].prod()
    tmp = tmp.rename('pred_for_team')
    
    
    em_train_df = pd.merge(em_train_df, tmp, on=['tour_id', 't_id', 'q_id'])
    tmp = em_train_df['pred_for_player'] / em_train_df['pred_for_team']
    
    
    em_train_df['z'] = tmp
    em_train_df['z'] = np.where(y_train == 1, tmp, 0)
    em_train_df['z'] = np.clip(em_train_df['z'], eps, 1 - eps)

    # M шаг.
    
    
    # https://stackoverflow.com/questions/42800769/scikit-learn-classification-on-soft-labels
    
    
    
    em_model.fit(X_train, logit(em_train_df['z']))
    em_train_df.drop(columns=['z', 'invers_pred_for_player'], inplace=True)
#     teams_id = em_train_df['t_id'].values
#     preds_for_team = em_train_df['pred_for_team'].values
    em_train_df['pred_for_player'] = expit(em_model.predict(X_train))
    em_train_df['pred_for_player'] = np.clip(em_train_df['pred_for_player'], eps, 1 - eps)
    em_train_df.drop(columns=['pred_for_team'], inplace=True)

    # Подсчет корреляций.
#     print("EM алгоритм. Шаг {}.".format(i))

#     team_rating_dict = {}
#     for t, r in zip(teams_id, preds_for_team):
#         team_rating_dict[t] = r
    em_players_force = em_model.coef_.ravel()[:len(em_train_df.p_id.unique())].astype(float)
    em_player_id_to_player_force_dict = {}
    for i, p_id in enumerate(ohe.categories_[0]):
        p_force = em_players_force[i]
        em_player_id_to_player_force_dict[p_id] = p_force

    em_spearman_corrs = []
    em_kendall_corrs = []

    for tour_id, teams in tour_id_to_teams_id_dict20.items():
        pred_ratings = []
        true_ratings = []
        for team_id in teams:
            true_res = tour_id_to_teams_results_dict20[tour_id][team_id]
            pred_res = 1
            players_cnt = 0
            for p_id in team_id_to_players_id_dict20[team_id]:
                if p_id in em_player_id_to_player_force_dict:
                    pred_res += em_player_id_to_player_force_dict[p_id]
                    players_cnt += 1
                else:
                    continue
            if players_cnt == 0:
                continue
            pred_res /= players_cnt
            pred_ratings.append(pred_res)
            true_ratings.append(true_res)

        em_spearman_corrs.append(stats.spearmanr(true_ratings, pred_ratings).correlation)
        em_kendall_corrs.append(stats.kendalltau(true_ratings, pred_ratings).correlation)

    spearman_metric = np.mean(np.array(em_spearman_corrs))
    kendall_metric = np.mean(np.array(em_kendall_corrs))

#     print("Корреляция Спирмена для EM-схемы: {}".format(spearman_metric))
#     print("Корреляция Кендалла для EM-схемы: {}".format(kendall_metric))
#     print()

    if best_metric < spearman_metric:
        best_metric = spearman_metric
        best_model = em_model
        
print("Корреляция Спирмена для EM-схемы: {}".format(best_metric))

Корреляция Спирмена для EM-схемы: 0.7522882612950267


Хммм, корреляция немного ухудшилась по сравнению с baseline...

## 5️⃣
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в [базе вопросов ЧГК](https://db.chgk.info/) (это чисто техническое дело, тут никакого ML нету).

In [74]:
len(ohe.categories_[1]) == len(questions_difficulty)

True

In [78]:
question_id_to_difficult_dict = {}

for i, q_id in tqdm(enumerate(ohe.categories_[1])):
    question_id_to_difficult_dict[q_id] = questions_difficulty[i]

29396it [00:00, 1442679.99it/s]


In [86]:
tour_id_diff = []

for tour_id, qs in tour_id_to_questions_dict19.items():
    d = 0
    for q_id in qs:
        d += question_id_to_difficult_dict[q_id]
    tour_id_diff.append({'tour_id': tour_id, 'tour_name':tour_id_name_dict[tour_id], 'difficulty': d})

tour_id_diff = sorted(tour_id_diff, key=lambda d: d['difficulty'], reverse=True)
tour_id_diff = pd.DataFrame(tour_id_diff)
tour_id_diff['rating_pos'] = tour_id_diff.index + 1

### Топ-20 легких турниров

In [87]:
tour_id_diff[:20]

,tour_id,tour_name,difficulty,rating_pos
0,5592,Студенческая лига ЧТ,151.715998,1
1,6008,Малый кубок Физтеха,139.223412,2
2,5660,Первая лига ЧТ,112.461543,3
3,6011,Щит и Меч,102.157978,4
4,5388,Олимпиада Приволжского федерального округа. Финал,94.296974,5
5,5488,Чемпионат Беларуси среди школьников,93.844232,6
6,5644,Школьный чемпионат России,93.682548,7
7,5658,Кубок Югры,93.619940,8
8,5457,Студенческий чемпионат Калининградской области,92.028708,9
9,5013,(а)Синхрон-lite. Лига старта. Эпизод V,89.268433,10


### Топ-20 трудных турниров

In [89]:
tour_id_diff[-20:]

,tour_id,tour_name,difficulty,rating_pos
619,6023,Гостиный двор,-58.382585,620
620,5083,Ускользающая сова,-58.666906,621
621,5693,Знание – Сила VI,-58.823826,622
622,5343,Кубок Физтеха,-60.512847,623
623,5993,Енисейская знать,-61.697280,624
624,5303,Мемориал Дмитрия Коноваленко,-62.007055,625
625,5773,Кубок Европы среди студентов,-62.316698,626
626,5587,Записки охотника,-62.700589,627
627,5368,AU,-64.657295,628
628,5903,Беловежская Зима,-65.280248,629


Видим, что среди трудных турниров различные крупные чемпионаты, а среди легких школьные и студенческие турниры.